In [37]:
from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import decomposition, ensemble
import numpy as np
import pandas, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn.metrics import confusion_matrix, accuracy_score

In [70]:
import pandas as pd 
data = pd.read_csv("/content/train.csv")

In [72]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [73]:
data1 = data[['text','target']]
data1.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [74]:
data1['text'] = data1['text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

data1['text'] = data1['text'].str.replace('[^\w\s]','')

data1['text'] = data1['text'].str.replace('\d','')

import nltk
nltk.download('omw-1.4')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('english')
data1['text'] = data1['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

sil = pd.Series(' '.join(data1['text']).split()).value_counts()[-1000:]
data1['text'] = data1['text'].apply(lambda x: " ".join(x for x in x.split() if x not in sil))

from textblob import Word
nltk.download('wordnet')
data1['text'] = data1['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This

In [75]:
data1.head()

,text,target
0,deed reason earthquake may allah forgive u,1
1,forest fire near la ronge sask canada,1
2,resident asked shelter place notified officer ...,1
3,people receive wildfire evacuation order calif...,1
4,got sent photo ruby alaska smoke wildfire pour...,1


In [76]:
data1.to_csv('data1.csv', index=False)

In [77]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(data1["text"],
                                                                     data1["target"],random_state=42)

In [78]:
train_x.head()

5151    dicehateme puppyshogun make sense paper beat r...
6351    catoinstitute cause federal failure deeply str...
3443    well chaning ipad screen fucking exploded glas...
7164                           war drug turned u war zone
7037     obama declares disaster typhoondevastated saipan
Name: text, dtype: object

In [79]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [80]:
train_x.head()

5151    dicehateme puppyshogun make sense paper beat r...
6351    catoinstitute cause federal failure deeply str...
3443    well chaning ipad screen fucking exploded glas...
7164                           war drug turned u war zone
7037     obama declares disaster typhoondevastated saipan
Name: text, dtype: object

In [81]:
train_y

array([0, 0, 0, ..., 0, 1, 1])

In [82]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer()

In [83]:
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

In [84]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(loj_model, 
                                           x_test_count, 
                                           test_y, 
                                           cv = 10).mean()

print("LogisticRegression Accuracy:", accuracy)

LogisticRegression Accuracy: 0.7678644254615596


In [85]:
y_pred = loj_model.predict(x_test_count)
accuracy_score(test_y, y_pred)

0.8019957983193278

In [86]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(loj_model, open(filename, 'wb'))
 

In [87]:
loaded_model = pickle.load(open(filename, 'rb'))

In [88]:
yeni_yorum = pd.Series("this film is very nice and good i like it")

yeni_yorum1 = pd.Series("there is a fire")

In [89]:
v = CountVectorizer()
v.fit(train_x)
yeni_yorum = v.transform(yeni_yorum)
yeni_yorum1 = v.transform(yeni_yorum1)

In [91]:
loaded_model.predict(yeni_yorum)

array([0])

In [90]:
loaded_model.predict(yeni_yorum1)

array([1])